In [15]:
import os
os.getcwd()

'/home/onyxia/work/hackathon-un-2022'

In [16]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic
import utils.export as export

In [17]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [18]:
def add_country(region, start_date, year, filter_cargo = False):
    ship_data_enriched = fc.create_ship_data_enriched()
    ais = fc.read_ais_all()
    ais_date = ais.loc[(ais["region"] == region) & (ais["start_date"] == start_date),:].copy()
    print(ais_date.shape)
    ais_enriched_date = fc.enrich_AIS_data(ais_date, ship_data_enriched)
    if filter_cargo:
        ais_enriched_date = fc.filter_cargo(ais_enriched_date)
    print(ais_enriched_date.shape)
    print(ais_enriched_date["mmsi"].nunique())
    ais_enriched_date, res_fuz_match_date = indic.fuzzy_match_destination_country(ais_enriched_date)
    res_fuz_match_date["year"] = year
    return res_fuz_match_date

In [19]:
res_fuz_match_19 = add_country("Black", "2019-04-01", "2019", filter_cargo=True)

(2331370, 8)
(1047588, 127)
222
Number of unique destination ports
1241
shape of ais
(1047588, 129)
shape of ais filtered with no na destination
(1018179, 129)
Fuzzy matching in process


/home/onyxia/work/hackathon-un-2022/utils/indicators.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["true_match"] = np.nan
/home/onyxia/work/hackathon-un-2022/utils/indicators.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["matched_destination_country"] = np.nan


Classification of destination countries
Norway                7
China                 6
United States         5
Italy                 5
Germany               4
Spain                 4
France                4
Saudi Arabia          4
Papua New Guinea      4
Jordan                4
Japan                 3
Sri Lanka             3
Sudan                 3
Turkey                3
Singapore             3
Egypt                 3
Ukraine               3
Yemen                 2
Russia                2
Algeria               2
Finland               2
Greece                2
Dominican Republic    2
Bangladesh            1
Morocco               1
Latvia                1
Argentina             1
Netherlands           1
Djibouti              1
Malaysia              1
Denmark               1
Greenland             1
Iran                  1
Canada                1
Jamaica               1
Tanzania              1
Venezuela             1
Chile                 1
Kenya                 1
Seychelles            1


In [ ]:
res_fuz_match_21 = add_country("Black", "2021-04-01", "2022", filter_cargo=True)

(2793423, 8)
(1445742, 127)
326
Number of unique destination ports
1319
shape of ais
(1445742, 129)


In [9]:
import pandas as pd
res_fuz_match = pd.concat([res_fuz_match_19, res_fuz_match_22])

In [10]:
res_fuz_match

,mmsi,matched_destination_country,matched_destination_port,ratio1,raw_port_destination,year
0,210385000,Ukraine,RENI,85.714286,BADAR EMAM KHOMIENI,2019
1,212175000,Singapore,KEPPEL - (EAST SINGAPORE),100.000000,SINGAPORE,2019
2,215015000,Romania,CONSTANTA,100.000000,CONSTANTA,2019
3,215122000,Equatorial Guinea,LUBA,85.714286,LUBUK GAUNG,2019
4,215137000,Nigeria,AGBAMI OIL TERMINAL,100.000000,T,2019
...,...,...,...,...,...,...
277,671700000,Mozambique,IBO,100.000000,DJIBOUTI,2021
278,677026300,Romania,MIDIA,100.000000,MIDIA_,2021
279,677030700,Saudi Arabia,JIDDAH,83.333333,JEDDAH,2021
280,677054000,Romania,MIDIA,100.000000,MIDIA,2021


In [14]:
export.export2minio("full_traces_destination_countries_by_mmsi_19_21.csv", res_fuz_match)

In [ ]:
export.export2minio("full_traces_onlycargo_destination_countries_by_mmsi_19_21.csv", res_fuz_match)# for filtered one

In [19]:
res_fuz_match_19 = add_country("Suez", "2019-04-01", "2019", filter_cargo=True)

(2331370, 8)
(1047588, 127)
222
Number of unique destination ports
1241
shape of ais
(1047588, 129)
shape of ais filtered with no na destination
(1018179, 129)
Fuzzy matching in process


/home/onyxia/work/hackathon-un-2022/utils/indicators.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["true_match"] = np.nan
/home/onyxia/work/hackathon-un-2022/utils/indicators.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["matched_destination_country"] = np.nan


Classification of destination countries
Norway                7
China                 6
United States         5
Italy                 5
Germany               4
Spain                 4
France                4
Saudi Arabia          4
Papua New Guinea      4
Jordan                4
Japan                 3
Sri Lanka             3
Sudan                 3
Turkey                3
Singapore             3
Egypt                 3
Ukraine               3
Yemen                 2
Russia                2
Algeria               2
Finland               2
Greece                2
Dominican Republic    2
Bangladesh            1
Morocco               1
Latvia                1
Argentina             1
Netherlands           1
Djibouti              1
Malaysia              1
Denmark               1
Greenland             1
Iran                  1
Canada                1
Jamaica               1
Tanzania              1
Venezuela             1
Chile                 1
Kenya                 1
Seychelles            1


In [20]:
res_fuz_match_21 = add_country("Suez", "2021-03-21", "2021", filter_cargo=True)

(2793423, 8)
(1445742, 127)
326
Number of unique destination ports
1319
shape of ais
(1445742, 129)
shape of ais filtered with no na destination
(1417854, 129)
Fuzzy matching in process


/home/onyxia/work/hackathon-un-2022/utils/indicators.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["true_match"] = np.nan
/home/onyxia/work/hackathon-un-2022/utils/indicators.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["matched_destination_country"] = np.nan


Classification of destination countries
Saudi Arabia            12
Singapore                8
Egypt                    7
United States            6
China                    6
Japan                    6
Sudan                    5
Indonesia                5
Netherlands              5
India                    5
Vietnam                  4
Djibouti                 4
Australia                4
Norway                   3
Jordan                   3
Ukraine                  3
Brazil                   3
Spain                    3
Yemen                    3
Romania                  3
South Korea              3
France                   3
Malaysia                 2
Papua New Guinea         2
Canada                   2
Israel                   2
United Arab Emirates     2
Belgium                  2
Italy                    2
Russia                   2
Taiwan                   1
Georgia                  1
Denmark                  1
Thailand                 1
Mexico                   1
Sri Lanka      

In [21]:
import pandas as pd
res_fuz_match = pd.concat([res_fuz_match_19, res_fuz_match_21])

In [22]:
res_fuz_match

,mmsi,matched_destination_country,matched_destination_port,ratio1,raw_port_destination,year
0,212175000,Singapore,KEPPEL - (EAST SINGAPORE),100.000000,SINGAPORE,2019
1,215015000,Romania,CONSTANTA,100.000000,CONSTANTA,2019
2,215137000,Seychelles,VICTORIA,100.000000,T,2019
3,215249000,Spain,VALENCIA,100.000000,VALENCIA,2019
4,220497000,France,OUISTREHAM,88.888889,DEHAM,2019
...,...,...,...,...,...,...
136,671700000,Djibouti,DJIBOUTI,100.000000,DJIBOUTI,2021
137,677026300,Romania,MIDIA,100.000000,MIDIA_,2021
138,677030700,Saudi Arabia,JIDDAH,83.333333,JEDDAH,2021
139,677054000,Romania,MIDIA,100.000000,MIDIA,2021


In [23]:
res_fuz_match.shape

(248, 6)

In [14]:
export.export2minio("full_traces_suez_destination_countries_by_mmsi_19_21.csv", res_fuz_match)
# for filtered one

In [24]:
export.export2minio("full_traces_suez_onlycargo_destination_countries_by_mmsi_19_21.csv", res_fuz_match)